Ref: https://alpaca.markets/learn/start-paper-trading

In [1]:
!pip install alpaca-py requests

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.3 requires pydantic<2.0, but you have pydantic 2.10.5 which is incompatible.



  Obtaining dependency information for alpaca-py from https://files.pythonhosted.org/packages/e7/c8/c10d049414682b6e0ec5616a743a7a3814dfbd8fd494161e757c1e247384/alpaca_py-0.35.1-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic<3.0.0,>=2.0.3 from https://files.pythonhosted.org/packages/58/26/82663c79010b28eddf29dcdd0ea723439535fa917fce5905885c0e9ba562/pydantic-2.10.5-py3-none-any.whl.metadata
  Obtaining dependency information for sseclient-py<2.0.0,>=1.7.2 from https://files.pythonhosted.org/packages/49/58/97655efdfeb5b4eeab85b1fc5d3fa1023661246c2ab2a26ea8e47402d4f2/sseclient_py-1.8.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for websockets>=10.4 from https://files.pythonhosted.org/packages/bc/ef/2d81679acbe7057ffe2308d422f744497b52009ea8bab34b6d74a2657d1d/websockets-14.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for annotated-types>=0.6.0 from https://files.pythonhosted.org/packages/78/b6/6307fbef88d9b5ee7421

# Authentication and Connecting to Alpaca’s Trading API

In [2]:
import requests
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

ALPACA_API_KEY_ID = os.getenv('ALPACA_API_KEY_ID')
ALPACA_API_SECRET_KEY = os.getenv('ALPACA_API_SECRET_KEY')

In [4]:
trading_client = TradingClient(ALPACA_API_KEY_ID, ALPACA_API_SECRET_KEY, paper=True)

In [6]:
url = "https://paper-api.alpaca.markets/v2/account"
headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": ALPACA_API_KEY_ID,
    "APCA-API-SECRET-KEY": ALPACA_API_SECRET_KEY
}
response = requests.get(url, headers=headers)
print(response.text)

{"id":"d5240e56-8a6b-4ece-bbf1-1cf0fbbfbfd8","admin_configurations":{},"user_configurations":null,"account_number":"PA3XHWVLV9KY","status":"ACTIVE","crypto_status":"ACTIVE","options_approved_level":3,"options_trading_level":3,"currency":"USD","buying_power":"200000","regt_buying_power":"200000","daytrading_buying_power":"0","effective_buying_power":"200000","non_marginable_buying_power":"100000","options_buying_power":"100000","bod_dtbp":"0","cash":"100000","accrued_fees":"0","portfolio_value":"100000","pattern_day_trader":false,"trading_blocked":false,"transfers_blocked":false,"account_blocked":false,"created_at":"2025-01-17T07:12:19.087835Z","trade_suspended_by_user":false,"multiplier":"2","shorting_enabled":true,"equity":"100000","last_equity":"100000","long_market_value":"0","short_market_value":"0","position_market_value":"0","initial_margin":"0","maintenance_margin":"0","last_maintenance_margin":"0","sma":"0","daytrade_count":0,"balance_asof":"2025-01-17","crypto_tier":1,"intrada

# Placing Paper Trades

In [8]:
order_symbol = "AAPL"     # 'META', 'AAPL', 'MSFT', 'AMZN', 'GOOG'
order_quantity = 1        #no.of shares
order_side = OrderSide.BUY    # OrderSide.BUY / OrderSide.SELL
order_time_in_force = TimeInForce.DAY

class TimeInForce(str, Enum):
    """
    Represents the various time in force options for an Order.

    The Time-In-Force values supported by Alpaca vary based on the order's security type. Here is a breakdown of the supported TIFs for each specific security type:
    - Equity trading: day, gtc, opg, cls, ioc, fok.
    - Options trading: day.
    - Crypto trading: gtc, ioc.
    Below are the descriptions of each TIF:
    - day: A day order is eligible for execution only on the day it is live. By default, the order is only valid during Regular Trading Hours (9:30am - 4:00pm ET). If unfilled after the closing auction, it is automatically canceled. If submitted after the close, it is queued and submitted the following trading day. However, if marked as eligible for extended hours, the order can also execute during supported extended hours.
    - gtc: The order is good until canceled. Non-marketable GTC limit orders are subject to price adjustments to offset corporate actions affecting the issue. We do not currently support Do Not Reduce(DNR) orders to opt out of such price adjustments.
    - opg: Use this TIF with a market/limit order type to submit “market on open” (MOO) and “limit on open” (LOO) orders. This order is eligible to execute only in the market opening auction. Any unfilled orders after the open will be cancelled. OPG orders submitted after 9:28am but before 7:00pm ET will be rejected. OPG orders submitted after 7:00pm will be queued and routed to the following day’s opening auction. On open/on close orders are routed to the primary exchange. Such orders do not necessarily execute exactly at 9:30am / 4:00pm ET but execute per the exchange’s auction rules.
    - cls: Use this TIF with a market/limit order type to submit “market on close” (MOC) and “limit on close” (LOC) orders. This order is eligible to execute only in the market closing auction. Any unfilled orders after the close will be cancelled. CLS orders submitted after 3:50pm but before 7:00pm ET will be rejected. CLS orders submitted after 7:00pm will be queued and routed to the following day’s closing auction. Only available with API v2.
    - ioc: An Immediate Or Cancel (IOC) order requires all or part of the order to be executed immediately. Any unfilled portion of the order is canceled. Only available with API v2. Most market makers who receive IOC orders will attempt to fill the order on a principal basis only, and cancel any unfilled balance. On occasion, this can result in the entire order being cancelled if the market maker does not have any existing inventory of the security in question.
    - fok: A Fill or Kill (FOK) order is only executed if the entire order quantity can be filled, otherwise the order is canceled. Only available with API v2.
    """

    DAY = "day"
    GTC = "gtc"
    OPG = "opg"
    CLS = "cls"
    IOC = "ioc"
    FOK = "fok"

In [9]:
# preparing market order
market_order_data = MarketOrderRequest(
                    symbol=order_symbol,
                    qty=order_quantity,
                    side=order_side,
                    time_in_force=order_time_in_force
                    )

# Market order
market_order = trading_client.submit_order(
                order_data=market_order_data
               )

In [ ]:
# Get a list of all of our orders.
portfolio = trading_client.get_orders()

# Print the quantity of shares for each order.
for position in portfolio:
    print("{} shares of {}".format(position.qty, position.symbol))

1 shares of AAPL
